# Using Open-source HuggingFace Models to Generate QAs from Raw Data

In this example, we will show you how to generate question-answers (QAs) from give text strings using open-source Huggingface models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

## Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

## Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

## Import dependency

In [2]:
from dotenv import load_dotenv
from IPython.display import display

from uniflow.client import Client
from uniflow.flow.flow_factory import FlowFactory
from uniflow.model.server import ModelServerFactory
from uniflow.config import HuggingfaceConfig
from uniflow.model.config import HuggingfaceModelConfig
from uniflow.viz import Viz

load_dotenv()

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare sample prompts

First, we need to demostrate sample prompts for LLM, those include instruction and sample json format. 

In [5]:
sample_instruction = """Generate one question and its corresponding answer based on the context. Following the format of the examples below to include context, question, and answer in the response."""

sample_json_format = [        
        {
            "context": """The quick brown fox jumps over the lazy dog.""",
            "question": """What is the color of the fox?""",
            "answer": """brown."""
        },
        {
            "context": """The quick brown fox jumps over the lazy black dog.""",
            "question": """What is the color of the dog?""",
            "answer": """black."""
        }]

Next, for the given raw text strings `raw_context_input` below, we can decorate them with our sample prompts. 

In [6]:
# raw_context_input = [
#     """We believe our success depends upon our capabilities in areas such as design, research and development, \
# production and marketing and is supported and protected by our intellectual property rights, such as trademarks, \
# utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for \
# and registering intellectual property rights in the United States and select foreign countries on trademarks, \
# inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our \
# intellectual property, including trademarks, patents and trade secrets against third-party infringement \
# and misappropriation.""",
#     """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of \
# information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our \
# journey here.""",
#     """The chain rule states that the derivative of a composite function (a function composed of another function) is equal to \
# the derivative of the outer function multiplied by the derivative of the inner function. Mathematically, it can be \
# written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
#     """Hypothesis testing involves making a claim about a population parameter based on sample data, and then conducting a test \
# to determine whether this claim is supported or rejected. This typically involves calculating a test statistic, \
# determining a significance level, and comparing the calculated value to a critical value to obtain a p-value. """
# ]


In [7]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously protect our intellectual property, including trademarks, patents and trade secrets against third-party infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) establishing the theory of information. In his article, Shannon introduced the concept of information entropy for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another function) is equal to the derivative of the outer function multiplied by the derivative of the inner function. Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then conducting a test to determine whether this claim is supported or rejected. This typically involves calculating a test statistic, determining a significance level, and comparing the calculated value to a critical value to obtain a p-value. """
]

raw_context_input_with_prompt = [
    {"instruction": sample_instruction, "examples": sample_json_format + [{"context": data}]}
    for data in raw_context_input
]

raw_context_input_with_prompt

[{'instruction': 'Generate one question and its corresponding answer based on the context. Following the format of the examples below to include context, question, and answer in the response.',
  'examples': [{'context': 'The quick brown fox jumps over the lazy dog.',
    'question': 'What is the color of the fox?',
    'answer': 'brown.'},
   {'context': 'The quick brown fox jumps over the lazy black dog.',
    'question': 'What is the color of the dog?',
    'answer': 'black.'},
   {'context': 'We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuabl

## Run model flow to generate data

TODO: figure out the optimal batchsize.

In [8]:
config = HuggingfaceConfig(model_config=HuggingfaceModelConfig(batch_size=4))
client = Client(config)

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.62s/it]


In [9]:
output = client.run(raw_context_input_with_prompt)

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 1/1 [00:13<00:00, 13.32s/it]


In [10]:
o = output[0]['output'][0]['response'][0]

In [11]:
import re
keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

segments = [segment for segment in re.split(pattern, o) if segment.strip()]
result = {
    "context": segments[-3],
    "question": segments[-2],
    "answer": segments[-1]
}


In [12]:
from pprint import pprint
pprint(result, sort_dicts=False)

{'context': ' We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party infringement and misappropriation.\n',
 'question': ' How does the company protect their intellectual property?\n',
 'answer': ' The company protects their intellectual property through various '
    